# Attention Architecture

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import music21

In [3]:
# from fastai.text import *
from enum import Enum
import torch
from fastai.text.models.awd_lstm import *
from fastai.text.models.transformer import *

In [4]:
import numpy as np
import torch.nn as nn

In [5]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [6]:
import sys
sys.path.insert(0, '../../')
from src.fastai_data import *
from src.encode_data import *
from src.serve import *

In [7]:
from src.lmnp_transformer import *

In [8]:
path = Path('../../data/midi/v15/piano_duet/')

## Single Stream Encoding

In [9]:
config = v15s_config(vocab); config

{'ctx_len': 150,
 'n_layers': 16,
 'n_heads': 8,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 512,
 'mask': True,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 16,
 'bptt': 256,
 'vocab_size': 274}

## Fastai Learner

In [10]:
dl_tfms = [mask_tfm, next_sentence_tfm]

In [11]:
data = load_music_data(path, cache_name='tmp/sample', vocab=vocab, y_offset=0, dl_tfms=dl_tfms, **config)

DLTFMS: [<function mask_tfm at 0x7f4ef013b8c8>, <function next_sentence_tfm at 0x7f4ef013b840>]


In [12]:
xb,yb = data.one_batch(cpu=False)

In [13]:
xb

tensor([[155,  68, 155,  ...,   8, 147,  52],
        [155,   4, 155,  ...,   8,   4,  52],
        [155,   4, 155,  ...,   8, 147,   4],
        ...,
        [155,  65, 155,  ...,   8, 147,  46],
        [ 53,  68, 155,  ...,   8, 147,   4],
        [155,  68, 155,  ...,   8, 147,  52]], device='cuda:0')

In [14]:
yb

[tensor([[  1,   1,   1,  ...,   1,   1,   1],
         [  1,  73,   1,  ...,   1, 147,   1],
         [  1,  72,   1,  ...,   1,   1,  48],
         ...,
         [  1,  65,   1,  ...,   1,   1,   1],
         [155,   1,   1,  ...,   1,   1,  49],
         [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0'),
 tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]], device='cuda:0')]

## LMNP

In [15]:
# m_len = 0
# x_len = 16 # bptt
# seq_len = m_len+x_len
# torch.triu(torch.ones(x_len, seq_len), diagonal=m_len).byte()[None,None].cpu().numpy()
# torch.triu(torch.ones(x_len, seq_len), diagonal=m_len+1).byte()[None,None].cpu().numpy()

In [16]:

class MusicTransformerXL(nn.Module):
    "Transformer model: https://arxiv.org/abs/1706.03762."
    def __init__(self, vocab_sz:int, ctx_len:int, n_layers:int, n_heads:int, d_model:int, d_head:int, d_inner:int, 
                 resid_p:float=0., attn_p:float=0., ff_p:float=0., embed_p:float=0., bias:bool=True, scale:bool=True,
                 act:Activation=Activation.ReLU, double_drop:bool=True, attn_cls:Callable=MultiHeadAttention,
                 learned_pos_enc:bool=True, mask:bool=True, **kwargs):
        super().__init__()
        self.mask = mask
        self.encoder = nn.Embedding(vocab_sz, d_model)
        self.pos_enc = nn.Embedding(ctx_len, d_model) if learned_pos_enc else PositionalEncoding(d_model)
        self.drop_emb = nn.Dropout(embed_p)
        self.layers = nn.ModuleList([DecoderLayer(n_heads, d_model, d_head, d_inner, resid_p=resid_p, attn_p=attn_p,
                      ff_p=ff_p, bias=bias, scale=scale, act=act, double_drop=double_drop, 
                      attn_cls=attn_cls) for k in range(n_layers)])
    
    def reset(self): pass
    
    def forward(self, x):
        bs, x_len = x.size()
        pos = torch.arange(0, x_len, device=x.device, dtype=x.dtype)
        inp = self.drop_emb(self.encoder(x) + self.pos_enc(pos)[None]) #.mul_(self.d_model ** 0.5)
        mask = torch.triu(x.new_ones(x_len, x_len), diagonal=1).byte()[None,None] if self.mask else None
        #[None,:,:None] for einsum implementation of attention
        for layer in self.layers: inp = layer(inp, mask=mask)
        return ([inp],[inp]) #For the LinearDecoder


In [17]:
# learn.model[0].layers[0].mhra.residual

In [18]:
from fastai.text.learner import _model_meta

In [19]:
config['mem_len'] = 0
config['mask'] = False
config['ctx_len'] = 512

In [20]:
config

{'ctx_len': 512,
 'n_layers': 16,
 'n_heads': 8,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 0,
 'mask': False,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 16,
 'bptt': 256,
 'vocab_size': 274}

In [21]:
_model_meta[MusicTransformerXL] = _model_meta[TransformerXL]
_model_meta[MusicTransformerXL]['config_lm'] = config

In [22]:

class LinearDecoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, output_p:float, tie_encoder:nn.Module=None, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1])
        decoded = self.decoder(output)
        return decoded, raw_outputs, outputs


In [23]:
class BertHead(nn.Module):
    def __init__(self, encoder, decoder, ns_decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.ns_decoder = ns_decoder
        
    def forward(self, x):
        x_e = self.encoder(x)
        return self.decoder(x_e), self.ns_decoder(x_e)
    
    def __getitem__(self, idx):
        return [self.encoder, self.decoder, self.ns_decoder][idx]
        
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in [self.encoder, self.decoder, self.ns_decoder]:
            if hasattr(c, 'reset'): c.reset()

In [24]:

class NSDecoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        if bias: self.decoder.bias.data.zero_()

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        raw_outputs, outputs = input
        decoded = self.decoder(outputs[-1])
        return decoded, raw_outputs, outputs

In [25]:
def get_music_model(arch:Callable, vocab_sz:int, config:dict=None, drop_mult:float=1.):
    "Create a language model from `arch` and its `config`, maybe `pretrained`."
    meta = _model_meta[arch]
    config = ifnone(config, meta['config_lm'].copy())
    for k in config.keys(): 
        if k.endswith('_p'): config[k] *= drop_mult
#     tie_weights,output_p,out_bias = map(config.pop, ['tie_weights', 'output_p', 'out_bias'])
    tie_weights,output_p,out_bias = map(config.get, ['tie_weights', 'output_p', 'out_bias'])
    init = config.pop('init') if 'init' in config else None
    encoder = arch(vocab_sz, **config)
    enc = encoder.encoder if tie_weights else None
    decoder = LinearDecoder(vocab_sz, config[meta['hid_name']], output_p, tie_encoder=enc, bias=out_bias)
    ns_decoder = NSDecoder(4, config[meta['hid_name']])
    model = BertHead(encoder, decoder, ns_decoder)
    return model if init is None else model.apply(init)


def music_model_learner(arch:Callable, data:DataBunch, config:dict=None, drop_mult:float=1., pretrained:bool=False,
                        pretrained_fnames:OptStrTuple=None, **learn_kwargs) -> 'LanguageLearner':
    "Create a `Learner` with a language model from `data` and `arch`."
    model = get_music_model(arch, config['vocab_size'], config=config, drop_mult=drop_mult)
    
    meta = _model_meta[arch]
    learn = MusicLearner(data, model, split_func=None, 
                         bos_idx=config['bos_idx'], sep_idx=config['sep_idx'],
                        **learn_kwargs)
    
    if pretrained:
        if 'url' not in meta: 
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta['url'], data=False)
        fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn.load_pretrained(*fnames)
        learn.freeze()
    if pretrained_fnames is not None:
        fnames = [learn.path/learn.model_dir/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]
        learn.load_pretrained(*fnames)
        learn.freeze()
    return learn

## Load

In [26]:
learn = music_model_learner(MusicTransformerXL, data, config, drop_mult=0)

Sep_idx: 8


In [27]:
xb,yb = data.one_batch(cpu=False)

In [28]:
# xb.shape, yb.shape

In [29]:
class BertLoss():
    def __init__(self, mask_loss, sent_loss):
        self.mask_loss = mask_loss
        self.sent_loss = sent_loss
        
    def __call__(self, input:Tensor, target:Tensor, target_sen:Tensor, **kwargs)->Rank0Tensor:
        m = self.mask_loss.__call__(input[0], target, **kwargs)
        s = self.sent_loss.__call__(input[1], target_sen, **kwargs)
        return m + s

In [30]:
from fastai.callbacks.rnn import RNNTrainer
class BertRNNTrainer(RNNTrainer):
    "`Callback` that regroups lr adjustment to seq_len, AR and TAR."
    def __init__(self, learn:Learner, alpha:float=0., beta:float=0.):
        super().__init__(learn, alpha, beta)
        
#     def on_epoch_begin(self, **kwargs):
#         "Reset the hidden state of the model."
#         self.learn.model.reset()

    def on_loss_begin(self, last_output:Tuple[Tensor,Tensor,Tensor], **kwargs):
        "Save the extra outputs for later and only returns the true output."
        return super().on_loss_begin(last_output[0])
#         self.raw_out,self.out = last_output[1],last_output[2]
#         return {'last_output': last_output[0]}

#     def on_backward_begin(self, last_loss:Rank0Tensor, last_input:Tensor, **kwargs):
#         "Apply AR and TAR to `last_loss`."
#         #AR and TAR
#         if self.alpha != 0.:  last_loss += self.alpha * self.out[-1].float().pow(2).mean()
#         if self.beta != 0.:
#             h = self.raw_out[-1]
#             if len(h)>1: last_loss += self.beta * (h[:,1:] - h[:,:-1]).float().pow(2).mean()
#         return {'last_loss': last_loss}

In [31]:
class BertTrainer(LearnerCallback):
    "`Callback` that regroups lr adjustment to seq_len, AR and TAR."
    def __init__(self, learn:Learner):
        super().__init__(learn)
        
    def on_loss_begin(self, last_output:Tuple[Tensor,Tensor,Tensor], **kwargs):
        "Save the extra outputs for later and only returns the true output."
        return {'last_output': (last_output[0][0], last_output[1][0]) }


In [32]:
type(learn.callbacks)

list

In [33]:
# learn.callbacks = [BertTrainer(learn, alpha=2, beta=1)]
learn.callbacks = [BertTrainer(learn)]

In [34]:
learn.loss_func = BertLoss(CrossEntropyFlat(ignore_index=vocab.pad_idx), CrossEntropyFlat())

In [35]:
learn.model(xb)

((tensor([[[-0.1301, -0.6223,  0.0786,  ..., -0.2835,  0.1422, -0.4134],
           [-0.3381, -0.4479,  0.4134,  ..., -0.3283, -0.0706, -0.0867],
           [ 0.0145,  0.4148,  0.1406,  ..., -0.0723, -0.5507, -0.3221],
           ...,
           [ 0.0192, -0.2977,  0.1053,  ..., -0.0690,  0.5979,  0.4539],
           [ 0.3925, -0.0971, -0.2308,  ..., -0.2477, -0.3514, -0.2282],
           [ 0.5407,  0.4429,  0.6588,  ..., -0.1949, -0.1373,  0.1382]],
  
          [[-0.1326, -0.6164,  0.0682,  ..., -0.2820,  0.1376, -0.4112],
           [-0.3394, -0.4547,  0.4058,  ..., -0.3296, -0.0651, -0.0921],
           [ 0.0141,  0.4151,  0.1405,  ..., -0.0723, -0.5508, -0.3222],
           ...,
           [ 0.0158, -0.2977,  0.0974,  ..., -0.0720,  0.5933,  0.4617],
           [ 0.4061, -0.0913, -0.2396,  ..., -0.2529, -0.3579, -0.2237],
           [ 0.5403,  0.4383,  0.6483,  ..., -0.1933, -0.1301,  0.1355]],
  
          [[-0.1270, -0.6277,  0.0812,  ..., -0.2772,  0.1389, -0.4079],
           

In [36]:
def mask_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    n = t1.shape[0]
    input = input[0].argmax(dim=-1).view(n,-1)
    t1 = t1.view(n,-1)
    mask = t1 != vocab.pad_idx
    return (input[mask]==t1[mask]).float().mean()

def ns_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    return accuracy(input[1], t2[0])

In [37]:
learn.metrics = [mask_acc, ns_acc]

In [38]:
learn.validate()

[7.1036386, tensor(0.0038), tensor(0.2199)]

In [39]:
# learn.lr_find()
# learn.recorder.plot()

In [40]:
learn.fit_one_cycle(3, 1e-4)

epoch,train_loss,valid_loss,mask_acc,ns_acc,time
0,4.230405,4.143900,0.250067,0.369497,01:55
1,4.166582,4.094497,0.188318,0.324444,01:55
2,4.105020,4.131458,0.248072,0.417856,01:55
